In [3]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
import csv, sqlite3

import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy.orm import Session, sessionmaker
import pandas as pd
import os


In [2]:
engine = create_engine("sqlite:///hawaii.sqlite")
connection = engine.connect()

In [4]:
Base = declarative_base()
Base.metadata.create_all(engine)

class Measurement(Base):
    __tablename__ = 'measurements'
    __table_args__ = {'sqlite_autoincrement':True}
    
    id = Column(Integer, primary_key=True)
    station = Column(String(50))
    date = Column(String(50))
    precipitation = Column(Integer)
    temperature = Column(Integer)


class Station(Base):
    __tablename__ = 'stations'
    __table_args__ = {'sqlite_autoincrement':True}
    
    id = Column(Integer, primary_key=True)
    station = Column(String(50))
    name = Column(String(50))
    latitude = Column(Integer)
    longitude = Column(Integer)
    elevation = Column(Integer)
    

In [5]:
Session = sessionmaker(bind=engine)
session = Session()

In [6]:
measurement_file = os.path.join("Resources","clean_hawaii_measurements.csv")
stations_file = os.path.join("Resources","clean_hawaii_stations.csv")
mdf = pd.read_csv(measurement_file)
sdf = pd.read_csv(stations_file)

In [7]:
sdf.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [9]:
mdf.to_sql(con=connection, index_label='id', name=Measurement.__tablename__, if_exists='replace')
sdf.to_sql(con=connection, index_label='id', name=Station.__tablename__, if_exists='replace')

In [10]:
for row in session.query(Measurement).limit(5).all():
    print("{}\t{}".format(row.station, row.precipitation))

USC00519397	0.08
USC00519397	0.0
USC00519397	0.0
USC00519397	0.0
USC00519397	0.06


In [12]:
for row in session.query(Station).limit(5).all():
    print("{}\t{}\t{}".format(row.station, row.name, row.elevation))

USC00519397	WAIKIKI 717.2, HI US	3.0
USC00513117	KANEOHE 838.1, HI US	14.6
USC00514830	KUALOA RANCH HEADQUARTERS 886.9, HI US	7.0
USC00517948	PEARL CITY, HI US	11.9
USC00518838	UPPER WAHIAWA 874.3, HI US	306.6


In [13]:
session.commit()
session.close()